# What drives the price of a car?

![](images/kurt.jpeg)

**OVERVIEW**

In this application, you will explore a dataset from kaggle. The original dataset contained information on 3 million used cars. The provided dataset contains information on 426K cars to ensure speed of processing.  Your goal is to understand what factors make a car more or less expensive.  As a result of your analysis, you should provide clear recommendations to your client -- a used car dealership -- as to what consumers value in a used car.

### CRISP-DM Framework

<center>
    <img src = images/crisp.png width = 50%/>
</center>


To frame the task, throughout our practical applications we will refer back to a standard process in industry for data projects called CRISP-DM.  This process provides a framework for working through a data problem.  Your first step in this application will be to read through a brief overview of CRISP-DM [here](https://mo-pcco.s3.us-east-1.amazonaws.com/BH-PCMLAI/module_11/readings_starter.zip).  After reading the overview, answer the questions below.

### Business Understanding

From a business perspective, we are tasked with identifying key drivers for used car prices.  In the CRISP-DM overview, we are asked to convert this business framing to a data problem definition.  Using a few sentences, reframe the task as a data task with the appropriate technical vocabulary. 

### Data Understanding

After considering the business understanding, we want to get familiar with our data.  Write down some steps that you would take to get to know the dataset and identify any quality issues within.  Take time to get to know the dataset and explore what information it contains and how this could be used to inform your business understanding.

In [724]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px

from sklearn.datasets import load_diabetes
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.metrics import mean_squared_error, r2_score
df = pd.read_csv('data/vehicles.csv')

In [725]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 426880 entries, 0 to 426879
Data columns (total 18 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   id            426880 non-null  int64  
 1   region        426880 non-null  object 
 2   price         426880 non-null  int64  
 3   year          425675 non-null  float64
 4   manufacturer  409234 non-null  object 
 5   model         421603 non-null  object 
 6   condition     252776 non-null  object 
 7   cylinders     249202 non-null  object 
 8   fuel          423867 non-null  object 
 9   odometer      422480 non-null  float64
 10  title_status  418638 non-null  object 
 11  transmission  424324 non-null  object 
 12  VIN           265838 non-null  object 
 13  drive         296313 non-null  object 
 14  size          120519 non-null  object 
 15  type          334022 non-null  object 
 16  paint_color   296677 non-null  object 
 17  state         426880 non-null  object 
dtypes: f

In [726]:
df.describe()

,id,price,year,odometer
count,4.268800e+05,4.268800e+05,425675.000000,4.224800e+05
mean,7.311487e+09,7.519903e+04,2011.235191,9.804333e+04
std,4.473170e+06,1.218228e+07,9.452120,2.138815e+05
min,7.207408e+09,0.000000e+00,1900.000000,0.000000e+00
25%,7.308143e+09,5.900000e+03,2008.000000,3.770400e+04
50%,7.312621e+09,1.395000e+04,2013.000000,8.554800e+04
75%,7.315254e+09,2.648575e+04,2017.000000,1.335425e+05
max,7.317101e+09,3.736929e+09,2022.000000,1.000000e+07


### Data Preparation

After our initial exploration and fine tuning of the business understanding, it is time to construct our final dataset prior to modeling.  Here, we want to make sure to handle any integrity issues and cleaning, the engineering of new features, any transformations that we believe should happen (scaling, logarithms, normalization, etc.), and general preparation for modeling with `sklearn`. 

In [727]:
#We will take a few steps to prepare the data for our modelling work. 
#1. Clean up NaN values from the dataframe.
#2. Delete columns that may not be relevant to the analysis, for example VIN number and id
#3. After exploring the data, we notice that the 'model' data is not very reliable as it has a wide array of models that are not standardized. We will remove it too

In [728]:
df['fuel'].unique()

array([nan, 'gas', 'other', 'diesel', 'hybrid', 'electric'], dtype=object)

In [729]:
df['drive'].unique()

array([nan, 'rwd', '4wd', 'fwd'], dtype=object)

In [730]:
df['model'].value_counts()

model
f-150                      8009
silverado 1500             5140
1500                       4211
camry                      3135
silverado                  3023
                           ... 
plymouth fury 1               1
f550 box truck delivery       1
duramax 2500hd                1
3 s sport                     1
Paige Glenbrook Touring       1
Name: count, Length: 29649, dtype: int64

In [731]:
df['transmission'].unique()

array([nan, 'other', 'automatic', 'manual'], dtype=object)

In [732]:
df['title_status'].unique()

array([nan, 'clean', 'rebuilt', 'lien', 'salvage', 'missing',
       'parts only'], dtype=object)

In [733]:
df['cylinders'].unique()

array([nan, '8 cylinders', '6 cylinders', '4 cylinders', '5 cylinders',
       'other', '3 cylinders', '10 cylinders', '12 cylinders'],
      dtype=object)

In [734]:
df['cylinders'].value_counts()

cylinders
6 cylinders     94169
4 cylinders     77642
8 cylinders     72062
5 cylinders      1712
10 cylinders     1455
other            1298
3 cylinders       655
12 cylinders      209
Name: count, dtype: int64

In [735]:
df.isnull().sum()

id                   0
region               0
price                0
year              1205
manufacturer     17646
model             5277
condition       174104
cylinders       177678
fuel              3013
odometer          4400
title_status      8242
transmission      2556
VIN             161042
drive           130567
size            306361
type             92858
paint_color     130203
state                0
dtype: int64

In [736]:
df = df.drop(columns=['VIN','id','region','state','model','paint_color'])
df

,price,year,manufacturer,condition,cylinders,fuel,odometer,title_status,transmission,drive,size,type
0,6000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,11900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,21000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
426875,23590,2019.0,nissan,good,6 cylinders,gas,32226.0,clean,other,fwd,NaN,sedan
426876,30590,2020.0,volvo,good,NaN,gas,12029.0,clean,other,fwd,NaN,sedan
426877,34990,2020.0,cadillac,good,NaN,diesel,4174.0,clean,other,NaN,NaN,hatchback
426878,28990,2018.0,lexus,good,6 cylinders,gas,30112.0,clean,other,fwd,NaN,sedan


In [737]:
df.isnull().sum()

price                0
year              1205
manufacturer     17646
condition       174104
cylinders       177678
fuel              3013
odometer          4400
title_status      8242
transmission      2556
drive           130567
size            306361
type             92858
dtype: int64

In [738]:
df.dropna(subset=['manufacturer','title_status','year','condition','cylinders','fuel','odometer','transmission','drive','size','type'], inplace = True)


In [739]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 83047 entries, 31 to 426836
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   price         83047 non-null  int64  
 1   year          83047 non-null  float64
 2   manufacturer  83047 non-null  object 
 3   condition     83047 non-null  object 
 4   cylinders     83047 non-null  object 
 5   fuel          83047 non-null  object 
 6   odometer      83047 non-null  float64
 7   title_status  83047 non-null  object 
 8   transmission  83047 non-null  object 
 9   drive         83047 non-null  object 
 10  size          83047 non-null  object 
 11  type          83047 non-null  object 
dtypes: float64(2), int64(1), object(9)
memory usage: 8.2+ MB


In [740]:
df.isnull().sum()

price           0
year            0
manufacturer    0
condition       0
cylinders       0
fuel            0
odometer        0
title_status    0
transmission    0
drive           0
size            0
type            0
dtype: int64

In [741]:
df = df[df['price'] > 500]
df = df[df['price']<195000]

In [742]:
df.nlargest(50, 'price')

,price,year,manufacturer,condition,cylinders,fuel,odometer,title_status,transmission,drive,size,type
104511,169999,2010.0,ferrari,excellent,8 cylinders,gas,13000.0,clean,automatic,rwd,full-size,coupe
113503,165000,2018.0,dodge,new,8 cylinders,gas,12.0,clean,automatic,rwd,full-size,coupe
323687,155000,2020.0,mercedes-benz,new,6 cylinders,diesel,250.0,clean,automatic,4wd,full-size,van
41657,150000,1959.0,jaguar,excellent,6 cylinders,gas,64765.0,clean,manual,rwd,compact,convertible
63153,150000,1959.0,jaguar,excellent,6 cylinders,gas,64765.0,clean,manual,rwd,compact,convertible
77020,150000,1959.0,jaguar,excellent,6 cylinders,gas,64765.0,clean,manual,rwd,compact,convertible
91502,150000,2009.0,ford,good,4 cylinders,gas,182415.0,clean,automatic,fwd,full-size,SUV
363978,150000,1959.0,jaguar,excellent,6 cylinders,gas,64765.0,clean,manual,rwd,compact,convertible
368720,144832,2013.0,volvo,good,5 cylinders,gas,144832.0,clean,automatic,rwd,mid-size,sedan
2922,140000,2015.0,jeep,like new,6 cylinders,gas,49658.0,rebuilt,other,fwd,mid-size,SUV


In [743]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 77562 entries, 31 to 426833
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   price         77562 non-null  int64  
 1   year          77562 non-null  float64
 2   manufacturer  77562 non-null  object 
 3   condition     77562 non-null  object 
 4   cylinders     77562 non-null  object 
 5   fuel          77562 non-null  object 
 6   odometer      77562 non-null  float64
 7   title_status  77562 non-null  object 
 8   transmission  77562 non-null  object 
 9   drive         77562 non-null  object 
 10  size          77562 non-null  object 
 11  type          77562 non-null  object 
dtypes: float64(2), int64(1), object(9)
memory usage: 7.7+ MB


In [744]:
df['transmission'].unique()

array(['automatic', 'manual', 'other'], dtype=object)

### Modeling

With your (almost?) final dataset in hand, it is now time to build some models.  Here, you should build a number of different regression models with the price as the target.  In building your models, you should explore different parameters and be sure to cross-validate your findings.

In [745]:
#First we will split our data for training and test, as well as removing the target feature (price) from the dataframe

In [746]:
from sklearn.preprocessing import LabelEncoder

# Initialize LabelEncoder
le = LabelEncoder()

# List of categorical columns to encode
categorical_columns = ["manufacturer", "condition", "fuel", "title_status", "transmission", "drive", "size", "type"]

# Apply label encoding to categorical columns
for col in categorical_columns:
    df[col] = le.fit_transform(df[col])

# Drop rows where cylinders is 'other'
df = df[df['cylinders'] != 'other']

# Convert cylinders to numeric
df['cylinders'] = df['cylinders'].str.extract('(\d+)').astype(int)
df

,price,year,manufacturer,condition,cylinders,fuel,odometer,title_status,transmission,drive,size,type
31,15000,2013.0,13,0,6,2,128000.0,0,0,2,1,10
55,19900,2004.0,13,2,8,0,88000.0,0,0,0,1,8
59,14000,2012.0,16,0,6,2,95000.0,0,0,1,1,5
65,22500,2001.0,13,2,8,0,144700.0,0,1,2,1,10
73,15000,2017.0,10,0,8,2,90000.0,4,0,2,2,9
...,...,...,...,...,...,...,...,...,...,...,...,...
426793,15999,2018.0,7,0,4,2,36465.0,0,0,1,2,9
426808,18000,2005.0,7,0,8,2,130000.0,1,0,0,1,10
426809,18000,1990.0,20,2,8,2,114400.0,0,0,0,1,0
426831,9800,1985.0,30,3,6,2,115000.0,0,0,2,3,4


In [747]:
df['cylinders'].unique()
df = df[df['cylinders']<10]
df = df[df['cylinders']>3]
#df = df[df['cylinders']!=5]

In [748]:
df['cylinders'].unique()

array([6, 8, 4, 5])

In [749]:
X = df.drop(columns=['price'])
X.info()

<class 'pandas.core.frame.DataFrame'>
Index: 76556 entries, 31 to 426833
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   year          76556 non-null  float64
 1   manufacturer  76556 non-null  int64  
 2   condition     76556 non-null  int64  
 3   cylinders     76556 non-null  int64  
 4   fuel          76556 non-null  int64  
 5   odometer      76556 non-null  float64
 6   title_status  76556 non-null  int64  
 7   transmission  76556 non-null  int64  
 8   drive         76556 non-null  int64  
 9   size          76556 non-null  int64  
 10  type          76556 non-null  int64  
dtypes: float64(2), int64(9)
memory usage: 7.0 MB


In [750]:
y = df['price']
y.info()

<class 'pandas.core.series.Series'>
Index: 76556 entries, 31 to 426833
Series name: price
Non-Null Count  Dtype
--------------  -----
76556 non-null  int64
dtypes: int64(1)
memory usage: 1.2 MB


In [751]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [752]:
models = {
    'Linear Regression': LinearRegression(),
    'Ridge Regression': Ridge(),
    'Lasso Regression': Lasso(),}

In [753]:
results = {}
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r2 = r2_score(y_test, y_pred)
    results[name] = {'RMSE': rmse, 'R2': r2}
    print(f"{name}: RMSE = {rmse:.4f}, R2 = {r2:.4f}")

Linear Regression: RMSE = 10493.0477, R2 = 0.2946
Ridge Regression: RMSE = 10493.0504, R2 = 0.2946
Lasso Regression: RMSE = 10493.0662, R2 = 0.2946


In [754]:
# Define the parameter grid for Ridge, Lasso, and Elastic Net
param_grid = {
    'Ridge': {'alpha': [0.1, 1.0, 10.0, 100.0]},
    'Lasso': {'alpha': [0.01, 0.1, 1.0, 10.0]}
}

# Initialize the models
models = {
    'Linear Regression': LinearRegression(),
    'Ridge': Ridge(),
    'Lasso': Lasso(),
}

# Perform cross-validation and hyperparameter tuning
best_models = {}
for name, model in models.items():
    if name in param_grid:
        grid = GridSearchCV(model, param_grid[name], cv=5, scoring='neg_mean_squared_error')
        grid.fit(X_train, y_train)
        best_models[name] = grid.best_estimator_
        best_rmse = np.sqrt(-grid.best_score_)
        print(f"Best {name}: {grid.best_params_}, RMSE = {best_rmse:.4f}")
    else:
        # For Linear Regression, just perform cross-validation
        scores = cross_val_score(model, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
        rmse_scores = np.sqrt(-scores)  # Convert negative MSE to RMSE
        best_models[name] = model
        print(f"{name}: Cross-validation RMSE = {rmse_scores.mean():.4f}")

print()
print('Evaluate the best models on the test set')
results = {}
for name, model in best_models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    rmse = mse ** 0.5
    r2 = r2_score(y_test, y_pred)
    results[name] = {'RMSE': rmse, 'R2': r2}
    print(f"{name}: RMSE = {rmse:.4f}, R2 = {r2:.4f}")

Linear Regression: Cross-validation RMSE = 10372.8179
Best Ridge: {'alpha': 10.0}, RMSE = 10374.2858
Best Lasso: {'alpha': 1.0}, RMSE = 10374.2859

Evaluate the best models on the test set
Linear Regression: RMSE = 10493.0477, R2 = 0.2946
Ridge: RMSE = 10493.0757, R2 = 0.2946
Lasso: RMSE = 10493.0662, R2 = 0.2946


In [759]:
# Use the best Linear model to make predictions
best_linear_model = best_models['Linear Regression']
y_pred = best_linear_model.predict(X_test)

# Create a DataFrame to show actual vs. predicted values
actual_vs_predicted_df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
actual_vs_predicted_df.head()

,Actual,Predicted
26078,7900,8439.306635
179066,4799,1089.738621
157631,33990,32925.378438
77946,6995,12174.238269
14063,12999,14503.080206


### Evaluation

With some modeling accomplished, we aim to reflect on what we identify as a high quality model and what we are able to learn from this.  We should review our business objective and explore how well we can provide meaningful insight on drivers of used car prices.  Your goal now is to distill your findings and determine whether the earlier phases need revisitation and adjustment or if you have information of value to bring back to your client.

In [760]:

# Get the coefficients and the intercept from the best Lasso model
coefficients = pd.Series(best_linear_model.coef_, index=X_train.columns)
intercept = best_linear_model.intercept_

print("Linear Regression Coefficients:")
print(coefficients)
print("\nIntercept:", intercept)

Linear Regression Coefficients:
year             363.693970
manufacturer      18.855220
condition         75.212238
cylinders       2553.935106
fuel           -5995.367777
odometer          -0.008133
title_status    -249.363372
transmission    3491.518990
drive           -865.213268
size            -235.095838
type               8.807758
dtype: float64

Intercept: -719126.1276115354



Generating learning curve for SFS_Ridge

Generating learning curve for Lasso

Generating learning curve for SFS_Linear
Learning curves have been generated and saved as PNG files.


In [761]:
from sklearn.inspection import permutation_importance
    
# Function to plot feature importance
def plot_feature_importance(importance, names, model_type):
    feature_importance = np.mean(importance, axis=1)
    feature_importance = 100.0 * (feature_importance / np.max(feature_importance))
    indices = np.argsort(feature_importance)
    
    plt.figure(figsize=(12, 8))
    plt.title(f'{model_type} Feature Importance')
    plt.barh(range(len(indices)), feature_importance[indices], align='center')
    plt.yticks(range(len(indices)), [names[i] for i in indices])
    plt.xlabel('Relative Importance')
    plt.tight_layout()
    plt.savefig(f'images/{model_type}_feature_importance.png')
    plt.close()

# Run permutation importance for each model
for model_name, model in models.items():
    print(f"\nRunning permutation importance for {model_name}")
    
    # Get best parameters from grid search
    best_params = results[f"{model_name}_KFold_5"]['best_params']
    
    # Create model with best parameters
    if model_name == 'SFS_Ridge':
        best_model = Pipeline([
            ('sfs', SequentialFeatureSelector(Ridge(), n_features_to_select=best_params['sfs__n_features_to_select'], direction=best_params['sfs__direction'])),
            ('ridge', Ridge(alpha=best_params['ridge__alpha']))
        ])
    elif model_name == 'Lasso':
        best_model = Lasso(alpha=best_params['alpha'])
    elif model_name == 'SFS_Linear':
        best_model = Pipeline([
            ('sfs', SequentialFeatureSelector(LinearRegression(), n_features_to_select=best_params['sfs__n_features_to_select'], direction=best_params['sfs__direction'])),
            ('lr', LinearRegression())
        ])
    
    # Fit the model
    best_model.fit(X_train, y_train)
    
    # Perform permutation importance
    perm_importance = permutation_importance(best_model, X_test, y_test, n_repeats=10, random_state=42, n_jobs=-1)
    
    # Get feature names (assuming X is a DataFrame, adjust if it's not)
    feature_names = X.columns.tolist()
    
    # Plot and save feature importance
    plot_feature_importance(perm_importance.importances, feature_names, model_name)
    
    # Print feature importance
    feature_importance = pd.DataFrame({
        'feature': feature_names,
        'importance': perm_importance.importances_mean
    }).sort_values('importance', ascending=False)
    
    print(f"\nFeature Importance for {model_name}:")
    print(feature_importance)


Running permutation importance for SFS_Ridge

Feature Importance for SFS_Ridge:
         feature  importance
3      cylinders    0.228034
0           year    0.179061
4           fuel    0.132756
5       odometer    0.035538
7   transmission    0.014172
8          drive    0.004468
6   title_status    0.001180
1   manufacturer    0.000912
9           size    0.000180
2      condition    0.000048
10          type    0.000000

Running permutation importance for Lasso

Feature Importance for Lasso:
         feature  importance
3      cylinders    0.227450
0           year    0.178474
4           fuel    0.132484
5       odometer    0.035530
7   transmission    0.013938
8          drive    0.004492
6   title_status    0.001130
1   manufacturer    0.000903
9           size    0.000160
2      condition    0.000042
10          type   -0.000051

Running permutation importance for SFS_Linear

Feature Importance for SFS_Linear:
         feature  importance
3      cylinders    0.228067
0        

### Deployment

Now that we've settled on our models and findings, it is time to deliver the information to the client.  You should organize your work as a basic report that details your primary findings.  Keep in mind that your audience is a group of used car dealers interested in fine tuning their inventory.